In [2]:
import pandas as pd
import dwsemear
import datetime
import collections
import numpy as np

import postgresemear as sql  # mover os arquivos para a pasta principal

def calcula_idade(dados, coluna):
    idade = []
    for i in dados.index:
        if dwsemear.e_nulo(dados.loc[i,coluna]):
            idade.append(None)
        else:
            anos = (dados.loc[i,'datahora'] - dados.loc[i,coluna]).days
            anos = int(round(anos/365))
            idade.append(anos)
    dados['idade'] = idade
    return dados

def remove_localize(data):
    return data.tz_localize(None)

def coluna_para_lista(coluna):
    nova_coluna = []
    for elem in coluna:
        if dwsemear.e_nulo(elem):
            nova_coluna.append(None)
        else:
            nova_coluna.append(elem)
    return nova_coluna

def para_int(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    else:
        return int(elemento)

def para_float(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    else:
        return float(elemento)

def para_str(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    else:
        return str(elemento)
    
def para_data(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    else:
        return elemento
    
def pegue_elementos_com_repeticao(coluna):
    contagem = collections.Counter(coluna)
    return list(getItemsUpToCount(contagem,1).keys())

def getItemsUpToCount(dct, n):
    upToCount = {}
    for key in dct:
        if dct[key] > n:
            upToCount[key] = dct[key]
    return upToCount


In [3]:
producao = pd.read_csv('BaseClientes-SET2019.csv',sep=';', encoding='latin-1',dtype=str)

In [5]:
producao = producao[producao.con_sta != 'C']
producao = producao.reset_index(drop=True)

In [7]:
producao = dwsemear.limpa_nome_das_colunas(producao)
producao.loc[:,'Con Contrato'] = producao.loc[:,'Con Contrato'].astype(int)
producao.loc[:,'DiasEmAtraso'] = producao.loc[:,'DiasEmAtraso'].astype(int)
producao.loc[:,'Pedido Jretail'] = producao.loc[:,'Pedido Jretail'].astype(int)

# Eliminando contratos que ja foram adicionados

In [63]:
# Testando
id_adicionados = sql.query_para_pandas('''SELECT ID
                      FROM producao''')

In [64]:
id_adicionados = set(id_adicionados.iloc[:,0].tolist())

In [66]:
producao.index = producao.loc[:,'Con Contrato']
intersec = list(id_adicionados.intersection(set(producao.index)))
producao = producao.drop(intersec)
producao = producao.reset_index(drop=True)

In [61]:
id_producao = 0
producao_registros = []
for i in producao.index:
    id_producao += 1
#    if id_producao > 10:
#        break

    producao_registros.append((para_int(producao.loc[i,'Con Contrato']),
                               para_int(producao.loc[i,'Pedido Jretail']),
                               para_int(producao.loc[i,'DiasEmAtraso'])))
                               
        
#    print('LINHA: ',
#          para_int(producao.loc[i,'Con Contrato']),
#          para_int(producao.loc[i,'Pedido Jretail']),  # esse cara poderia ser not null
#          para_int(producao.loc[i,'DiasEmAtraso']))


In [62]:
query = """ INSERT INTO producao
            (ID,
            codigo_proposta_crivo,
            dias_atraso) 
            VALUES (%s,%s,%s)"""
sql.adicione_multiplos_registros(query, producao_registros)

In [8]:
# Testando
parametros = [1,31]
dados_producao = sql.query_para_pandas('''SELECT *
                      FROM producao
                      WHERE ID BETWEEN %s AND %s
                      ''',parametros)